# Emotions Recognition AI Project


### 사용할 모델 다운로드
#### face-detection-adas-0001 다운로드

In [1]:
! omz_downloader --name face-detection-adas-0001 --precision FP16

################|| Downloading face-detection-adas-0001 ||################

========== Downloading C:\BrainAI\emotions-recognition\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.xml
... 100%, 304 KB, 329 KB/s, 0 seconds passed

========== Downloading C:\BrainAI\emotions-recognition\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.bin
... 49%, 1024 KB, 744 KB/s, 1 seconds passed
... 99%, 2048 KB, 1272 KB/s, 1 seconds passed
... 100%, 2056 KB, 1278 KB/s, 1 seconds passed



## emotions-recognition-retail-0003 모델 다운로드

In [2]:
! omz_downloader --name emotions-recognition-retail-0003 --precision FP16

################|| Downloading emotions-recognition-retail-0003 ||################

========== Downloading C:\BrainAI\emotions-recognition\intel\emotions-recognition-retail-0003\FP16\emotions-recognition-retail-0003.xml
... 100%, 54 KB, 3624 KB/s, 0 seconds passed

========== Downloading C:\BrainAI\emotions-recognition\intel\emotions-recognition-retail-0003\FP16\emotions-recognition-retail-0003.bin
... 21%, 1024 KB, 172 KB/s, 5 seconds passed
... 42%, 2048 KB, 279 KB/s, 7 seconds passed
... 63%, 3072 KB, 372 KB/s, 8 seconds passed
... 84%, 4096 KB, 458 KB/s, 8 seconds passed
... 100%, 4848 KB, 525 KB/s, 9 seconds passed



## 필요한 라이브러리 Imports

In [3]:
import openvino as ov
import cv2
import numpy as np

import matplotlib.pyplot as plt
from pathlib import Path

### Inference 할 device 확인

In [4]:
core = ov.Core()
options=core.available_devices

options

['CPU', 'GPU']

### Load the Model

In [5]:
#core = ov.Core()

model = core.read_model(model='./models/face-detection-adas-0001.xml')
face_model = core.compile_model(model=model, device_name="CPU")

face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)

print("Input layer shape: ", face_input_layer.shape)
print("Output layer shape:", face_output_layer.shape)

Input layer shape:  [1,3,384,672]
Output layer shape: [1,1,200,7]


In [6]:
model = core.read_model(model='./models/emotions-recognition-retail-0003.xml')
emotion_model = core.compile_model(model=model, device_name="CPU")

emotion_input_layer = emotion_model.input(0)
emotion_output_layer = emotion_model.output(0)

print("Input layer shape: ", emotion_input_layer.shape)
print("Output layer shape:", emotion_output_layer.shape)

Input layer shape:  [1,3,64,64]
Output layer shape: [1,5,1,1]


### Load Image

In [34]:
frame = cv2.imread("images/test.jpeg")

resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
transposed_frame = resized_frame.transpose(2, 0, 1)
input_frame = np.expand_dims(transposed_frame, 0)

### DrawBouningBoxes() 함수 만들기

In [35]:
def DrawBoundingBoxes(output, frame, conf=0.5):
    boxes = []
    canvas = frame.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    for detection in top_predictions:
        box = (detection[3:7] * np.array([w, h, w, h])).astype("int") # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box   # xmin, ymin, xmax, ymax에 상자 위치 값 지정
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 그리기
        boxes.append(box)     #이미지에 박스를 그린 얼굴의 위치 저장
   
    return boxes

In [45]:
def DrawText(output, frame, face_position):
    # emotions 딕셔너리 생성
    emotions = {
        0:"neutral",
        1:"happy",
        2:"sad",
        3:"surprise",
        4:"anger"
    }
    # 딕셔너리 출력하기
    #for key, value in emotions.items():
    #    print(key, value, end='      ')
    #print()
        
    predictions = output[0,:,0,0]              # 5개의 감정 예측값 저장
    #print("predictions : " + str(predictions))
    
    topresult_index = np.argmax(predictions)   # 5개의 감정 예측값 중 가장 높은값의 위치 저장
    #print("topresult_index : " + str(topresult_index))
    
    emotion = emotions[topresult_index]        # emotions에서 topresult_index 값에 해당하는 감정 저장
    #print("emotion : " + emotion)
    
    cv2.putText(frame, emotion,                 # 예측한 감정값 이미지에 출력하기
                (face_position[0],face_position[1]),    #xmin, ymin 값을 가져와 위치 설정
                cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (0, 0,255), 2)

In [46]:
face_output = face_model([input_frame])[face_output_layer]
boxes = DrawBoundingBoxes(face_output, frame, conf=0.5)

In [47]:
if boxes is not None:
    
    for box in boxes:          #boxes에 저장된 얼굴의 위치들을 하나씩 box에 전달
    
        xmin, ymin, xmax, ymax = box      #box에 저장된 좌표 저장
        emotion_input = frame[ymin:ymax,xmin:xmax]         #이미지에서 해당 얼굴 위치를 찾아 저장
        print(emotion_input.shape)
        
        # 감정 인식 모델을 사용하기 위해 이미지 전처리
        # Input layer shape:  [1,3,64,64]
        resized_image = cv2.resize(src=emotion_input, dsize=(64, 64))      #이미지 사이즈 변경  (64,64,3)
        transposed_image = resized_image.transpose(2, 0, 1)                #shape 위치 변경    (3,64,64)
        input_image = np.expand_dims(transposed_image, 0)                  #차원 확장 (1,3,64,64)

        emotion_output = emotion_model([input_image])[emotion_output_layer]  # 감정 추론
        DrawText(emotion_output, frame, box)   # 추론의 결과값 이미지에 출력하기

(203, 159, 3)


In [39]:
cv2.imshow("emotion-recognition", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
def AddBackground(frame, bg):

    frame_h, frame_w = frame.shape[0], frame.shape[1]
    new_h = 500
    new_w = int((new_h/frame_h)*frame_w)
    frame_resize = cv2.resize(frame, (new_w, new_h))

    xmax = bg.shape[1] - 400
    ymax = bg.shape[0] - 175
    xmin = xmax - new_w
    ymin = ymax - new_h

    bg[ymin:ymax, xmin:xmax] = frame_resize

    return bg

In [49]:
background = "./images/Background2.jpg"  #사용할 배경화면 경로

bg = cv2.imread(background)
deployment = AddBackground(frame, bg)

In [50]:
cv2.imshow("Deployment", deployment)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [51]:
camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)
background = "./images/Background2.jpg"
bg = cv2.imread(background)

while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    if ret == False:
        break
    
    resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0)    
    
    face_output = face_model([input_frame])[face_output_layer]
    
    boxes = DrawBoundingBoxes(face_output, frame, conf=0.5)
    
    if boxes is not None:
    
        for box in boxes:          #boxes에 저장된 얼굴의 위치들을 하나씩 box에 전달
    
            xmin, ymin, xmax, ymax = box      #box에 저장된 좌표 저장
            emotion_input = frame[ymin:ymax,xmin:xmax]         #이미지에서 해당 얼굴 위치를 찾아 저장
        
            # 감정 인식 모델을 사용하기 위해 이미지 전처리
            # Input layer shape:  [1,3,64,64]
            resized_image = cv2.resize(src=emotion_input, dsize=(64, 64))      #이미지 사이즈 변경  (64,64,3)
            transposed_image = resized_image.transpose(2, 0, 1)                #shape 위치 변경    (3,64,64)
            input_image = np.expand_dims(transposed_image, 0)                  #차원 확장 (1,3,64,64)

            emotion_output = emotion_model([input_image])[emotion_output_layer]  # 감정 추론
            DrawText(emotion_output, frame, box)   # 추론의 결과값 이미지에 출력하기
    
    deployment = AddBackground(frame, bg)
    
    cv2.imshow('Press Spacebar to Exit', deployment)

    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break

camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()